In [1]:
import numpy as np
import matplotlib.pyplot as plt

Satelli sampling means variance based sampling, but not a good measure for non-uniform distribution. Advantage: just extends series further if you want to increase number of samples
Parallelization of sampling --> batch run (concurrent)

# Added modules

In [ ]:
def get_theta(x, mu, sigma):
    """
    x: fraction of similar neighbours
    mu: optimal fraction of similar neighbours
    sigma: acceptance range
    """
    theta = np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))
    return theta

In [ ]:
def gaussian_function(x, mu, sigma):
    return np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))

# Example usage:
x_values = np.linspace(0, 1, 100)
mu = 0.5  # Peak in the middle
sigma = 0.6  # Controls the width

y_values = gaussian_function(x_values, mu, sigma)

# You can plot the function to visualize it
import matplotlib.pyplot as plt

plt.plot(x_values, y_values)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title(f'Gaussian Function with Peak at {mu}')
plt.show()

# Sobol Sensitivity Analysis

In [2]:
from SALib.sample import saltelli
from SALib.analyze import sobol

In [3]:
import modules as modules
import model as model

# # Create and run the model
model = model.Schelling(
     property_value_func=modules.property_value_quadrants,
     income_func=modules.income_func,
     desirability_func=modules.desirability_func,
     utility_func=modules.utility_func,
     price_func=modules.price_func,
     compute_similar_neighbours=modules.compute_similar_neighbours,
     height=20,
     width=20,
     radius=1,
     density=0.8,
     minority_pc=0.2,
     alpha=0.5,
     seed=42
 )

/Users/kayamarlenedisselkamp/Desktop/Master - Computational Science/Agent-Based Modelling/ABM-Notebooks-0-1/ABM_Group_3/Mesa Schelling Housing/modules.py:23: UserWarning: Default value 0 (int) might not be best suitable with dtype=float64.
  layer = mesa.space.PropertyLayer(name, width, height, 0)
/Users/kayamarlenedisselkamp/Desktop/Master - Computational Science/Agent-Based Modelling/ABM-Notebooks-0-1/ABM_Group_3/Mesa Schelling Housing/modules.py:23: FutureWarning: The new PropertyLayer and _PropertyGrid classes experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1932
  layer = mesa.space.PropertyLayer(name, width, height, 0)
/Users/kayamarlenedisselkamp/Desktop/Master - Computational Science/Agent-Based Modelling/ABM-Notebooks-0-1/ABM_Group_3/Mesa Schelling Housing/model.py:140

In [6]:
# # Run the model for a certain number of steps
for i in range(5):
     #print(i)
     #print(model.entropy)
     model.step()

# Exemplory use of accessing datacollector
model_data = model.datacollector.get_model_vars_dataframe()
agent_data = model.datacollector.get_agent_vars_dataframe()
print(model_data)
print(agent_data)

                                         Desirability
0   [[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,...
1   [[0.6091614906832298, 0.6091614906832298, 0.60...
2   [[0.05854037267080746, 0.23462732919254659, 0....
3   [[0.06692546583850932, 0.2849378881987578, 0.4...
4   [[0.06413043478260869, 0.27096273291925466, 0....
5   [[0.07251552795031056, 0.28214285714285714, 0....
6   [[0.07531055900621117, 0.27375776397515533, 0....
7   [[0.08090062111801243, 0.27375776397515533, 0....
8   [[0.06972049689440994, 0.25978260869565223, 0....
9   [[0.06972049689440994, 0.2625776397515528, 0.4...
10  [[0.06972049689440994, 0.2653726708074534, 0.4...
11  [[0.07531055900621117, 0.2653726708074534, 0.4...
12  [[0.0781055900621118, 0.2625776397515528, 0.43...
13  [[0.0781055900621118, 0.2653726708074534, 0.43...
14  [[0.08090062111801243, 0.2653726708074534, 0.4...
15  [[0.08369565217391306, 0.2653726708074534, 0.4...
               Utility  Segregation  Moves
Step AgentID                           

In [7]:
# Step 1: Problem definition
problem = {
    'num_vars': 6,
    'names': ['density', 'minority_pc', 'property_value_weight', 'alpha', 'mu_theta', 'sigma_theta'],
    'bounds': [[0,1], [0,1], [0,1], [0,1], [0,1], [0,1]]
}

In [8]:
# Step 2: Generate the samples (power of 2 for best performance)
samples = saltelli.sample(problem, 1) #2**5) #2024

/var/folders/v9/l1nzr55125v697bcr39l01hr0000gn/T/ipykernel_51133/3161514562.py:2: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  samples = saltelli.sample(problem, 1) #2**5) #2024
/Users/kayamarlenedisselkamp/.local/share/virtualenvs/ABM-Notebooks-0-1-OzA5lFMJ/lib/python3.11/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (1) is equal to `2^n`.
        
  warnings.warn(msg)


In [9]:
# Step 3: Run the model for generated samples
import model
import modules

# First define the model such that it runs for a specified number of time steps in a function 
def run_schelling_model(property_value_func,
                        income_func,
                        desirability_func,
                        utility_func,
                        price_func,
                        compute_similar_neighbours,
                        height,
                        width,
                        radius,
                        params,
                        #density,
                        #minority_pc,
                        #alpha,
                        #property_value_weight,
                        #mu_theta,
                        #sigma_theta,
                        seed,
                        num_steps):
    # Initialize the model
    model_instance = model.Schelling(
        property_value_func=property_value_func,
        income_func=income_func,
        desirability_func=desirability_func,
        utility_func=utility_func,
        price_func=price_func,
        compute_similar_neighbours=compute_similar_neighbours,
        height=height,
        width=width,
        radius=radius,
        density=params[0], #density,
        minority_pc=params[1], #minority_pc,
        alpha=params[2], #alpha,
        income_scale=1.5, # the scale by which the income is higher than the property value
        property_value_weight=params[3], #property_value_weight,
        mu_theta=params[4], #mu_theta,
        sigma_theta=params[5],
        seed=seed
    )

    # Run the model for the specified number of steps and collect entropy values
    entropies = []
    for _ in range(num_steps):
        model_instance.step()
        entropies.append(model_instance.entropy)

    # Compute mean and standard deviation of entropy
    entropy_mean = np.mean(entropies)
    entropy_std = np.std(entropies)

    # 
    model_data = model_instance.datacollector.get_model_vars_dataframe()
    agent_data = model_instance.datacollector.get_agent_vars_dataframe()

    # Return the model instance and entropy values
    return model_instance, np.array(entropies), entropy_mean, entropy_std, model_data, agent_data

# Exemplory use
"""# Example of calling the function with specific parameters
model_result = run_schelling_model(
    property_value_func=modules.property_value_quadrants,
    income_func=modules.income_func,
    desirability_func=modules.desirability_func,
    utility_func=modules.utility_func,
    price_func=modules.price_func,
    compute_similar_neighbours=modules.compute_similar_neighbours,
    height=20,
    width=20,
    radius=1,
    density=0.8,
    minority_pc=0.2,
    alpha=0.5,
    seed=42,
    num_steps=5
)"""

'# Example of calling the function with specific parameters\nmodel_result = run_schelling_model(\n    property_value_func=modules.property_value_quadrants,\n    income_func=modules.income_func,\n    desirability_func=modules.desirability_func,\n    utility_func=modules.utility_func,\n    price_func=modules.price_func,\n    compute_similar_neighbours=modules.compute_similar_neighbours,\n    height=20,\n    width=20,\n    radius=1,\n    density=0.8,\n    minority_pc=0.2,\n    alpha=0.5,\n    seed=42,\n    num_steps=5\n)'

In [10]:
samples[0]

array([0.09375, 0.46875, 0.46875, 0.65625, 0.28125, 0.96875])

In [11]:
# Next compute the model for the different parameter settings 
Y = run_schelling_model(property_value_func=modules.property_value_quadrants,
    income_func=modules.income_func,
    desirability_func=modules.desirability_func,
    utility_func=modules.utility_func,
    price_func=modules.price_func,
    compute_similar_neighbours=modules.compute_similar_neighbours,
    height=20,
    width=20,
    radius=1,
    params=samples[0],
    seed=42,
    num_steps=2)

/Users/kayamarlenedisselkamp/Desktop/Master - Computational Science/Agent-Based Modelling/ABM-Notebooks-0-1/ABM_Group_3/Mesa Schelling Housing/modules.py:23: UserWarning: Default value 0 (int) might not be best suitable with dtype=float64.
  layer = mesa.space.PropertyLayer(name, width, height, 0)
/Users/kayamarlenedisselkamp/Desktop/Master - Computational Science/Agent-Based Modelling/ABM-Notebooks-0-1/ABM_Group_3/Mesa Schelling Housing/model.py:140: UserWarning: Default value 0.5 (float) might not be best suitable with dtype=float64.
  self.desirability_layer = mesa.space.PropertyLayer("desirability", width, height, 0.5)
/Users/kayamarlenedisselkamp/Desktop/Master - Computational Science/Agent-Based Modelling/ABM-Notebooks-0-1/ABM_Group_3/Mesa Schelling Housing/model.py:146: UserWarning: Default value 0 (int) might not be best suitable with dtype=float64.
  self.interested_agents_layer = mesa.space.PropertyLayer("interested_agents", width, height, 0)
/Users/kayamarlenedisselkamp/Desk

In [12]:
Y[3]

0.05690927742243401

In [ ]:
import numpy as np
from SALib.sample import saltelli
from SALib.analyze import sobol

# Step 1: Define the problem
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[0, 1], [0, 1], [0, 1]]
}

# Step 2: Generate samples
param_values = saltelli.sample(problem, 1000)

# Step 3: Evaluate the model
# Define your model (this is an example model)
def model(X):
    x1 = X[:, 0]
    x2 = X[:, 1]
    x3 = X[:, 2]
    return x1 + x2 + x3

# Run the model for generated samples
Y = model(param_values)

In [ ]:
print(Y)

In [ ]:
problem

In [ ]:
# Step 4: Perform Sobol sensitivity analysis
sobol_indices = sobol.analyze(problem, Y, print_to_console=True)

# Step 5: Extract results
Si = sobol_indices

# Print the first-order, second-order, and total-order sensitivity indices
print("First-order indices:", Si['S1'])
print("Second-order indices:", Si['S2'])
print("Total-order indices:", Si['ST'])